# Day 5: Particle Filters for Nonlinear/Non-Gaussian Systems

## Week 20: Bayesian Methods in Finance

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why Kalman filters fail for nonlinear/non-Gaussian systems
2. **Master** Sequential Monte Carlo (SMC) fundamentals
3. **Implement** Bootstrap Particle Filter from scratch
4. **Apply** resampling strategies (Multinomial, Systematic, Stratified)
5. **Build** particle filters for financial applications (stochastic volatility, regime switching)
6. **Diagnose** particle degeneracy and implement solutions

---

## 📚 Table of Contents

1. [Motivation: Beyond Kalman Filters](#1-motivation)
2. [Sequential Monte Carlo Fundamentals](#2-smc-fundamentals)
3. [The Bootstrap Particle Filter](#3-bootstrap-pf)
4. [Resampling Strategies](#4-resampling)
5. [Financial Applications](#5-finance-applications)
6. [Advanced Topics: SIR, Auxiliary PF, Rao-Blackwellization](#6-advanced)
7. [Interview Questions & Exercises](#7-interview)

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import logsumexp
import warnings
warnings.filterwarnings('ignore')

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Environment ready for Particle Filter analysis!")

---

## 1. Motivation: Beyond Kalman Filters <a name="1-motivation"></a>

### 1.1 The Filtering Problem

**State-Space Model:**

$$\begin{align}
x_t &= f(x_{t-1}, u_t) \quad \text{(State Transition)} \\
y_t &= h(x_t, v_t) \quad \text{(Observation)}
\end{align}$$

Where:
- $x_t$ = hidden state (e.g., true volatility, regime)
- $y_t$ = observation (e.g., returns, prices)
- $u_t, v_t$ = noise terms (not necessarily Gaussian!)

**Goal:** Compute the posterior distribution $p(x_t | y_{1:t})$

### 1.2 When Kalman Filters Fail

| Assumption | Kalman Filter | Particle Filter |
|------------|---------------|------------------|
| State transition | Linear | **Any nonlinear** |
| Observation model | Linear | **Any nonlinear** |
| Noise distribution | Gaussian | **Any distribution** |
| Posterior | Gaussian (exact) | **Approximated by particles** |

**Financial Examples Where KF Fails:**
1. **Stochastic Volatility Models** - Volatility evolves nonlinearly
2. **Regime-Switching Models** - Discrete state space
3. **Jump-Diffusion Models** - Non-Gaussian innovations
4. **Option Pricing** - Nonlinear observation functions

In [ ]:
# Demonstration: Why Kalman Filters Fail for Nonlinear Systems

def demonstrate_kalman_failure():
    """
    Show a simple nonlinear system where Kalman filter fails.
    Model: x_t = 0.5*x_{t-1} + 25*x_{t-1}/(1+x_{t-1}^2) + 8*cos(1.2*t) + w_t
           y_t = x_t^2/20 + v_t
    """
    T = 100
    Q = 10  # Process noise variance
    R = 1   # Observation noise variance
    
    # True states and observations
    x_true = np.zeros(T)
    y = np.zeros(T)
    x_true[0] = 0.1
    
    for t in range(1, T):
        # Nonlinear state transition
        x_true[t] = (0.5 * x_true[t-1] + 
                     25 * x_true[t-1] / (1 + x_true[t-1]**2) + 
                     8 * np.cos(1.2 * t) + 
                     np.sqrt(Q) * np.random.randn())
    
    # Nonlinear observation
    y = x_true**2 / 20 + np.sqrt(R) * np.random.randn(T)
    
    # Naive Kalman Filter (treating as linear - WRONG!)
    x_kf = np.zeros(T)
    P = np.zeros(T)
    x_kf[0] = 0
    P[0] = 1
    
    for t in range(1, T):
        # Linearized (incorrectly) prediction
        x_pred = 0.5 * x_kf[t-1]  # Missing nonlinear terms!
        P_pred = 0.25 * P[t-1] + Q
        
        # Linearized update (also wrong)
        H = x_pred / 10  # Linearized observation Jacobian
        K = P_pred * H / (H**2 * P_pred + R)
        x_kf[t] = x_pred + K * (y[t] - x_pred**2/20)
        P[t] = (1 - K * H) * P_pred
    
    # Plot results
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    axes[0].plot(x_true, 'b-', lw=2, label='True State', alpha=0.8)
    axes[0].plot(x_kf, 'r--', lw=2, label='Kalman Filter (Linearized)', alpha=0.8)
    axes[0].fill_between(range(T), x_kf - 2*np.sqrt(P), x_kf + 2*np.sqrt(P), 
                         alpha=0.2, color='red', label='±2σ Confidence')
    axes[0].set_xlabel('Time')
    axes[0].set_ylabel('State')
    axes[0].set_title('Kalman Filter Fails for Nonlinear System')
    axes[0].legend()
    
    axes[1].plot(y, 'g.', alpha=0.5, label='Observations')
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Observation')
    axes[1].set_title('Nonlinear Observations: y = x²/20 + noise')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    rmse = np.sqrt(np.mean((x_true - x_kf)**2))
    print(f"\n📊 Kalman Filter RMSE: {rmse:.2f}")
    print("⚠️ High error due to nonlinearity - Particle Filter needed!")
    
    return x_true, y, Q, R

x_true, y_obs, Q, R = demonstrate_kalman_failure()

---

## 2. Sequential Monte Carlo Fundamentals <a name="2-smc-fundamentals"></a>

### 2.1 The Key Idea

**Represent the posterior with weighted samples (particles):**

$$p(x_t | y_{1:t}) \approx \sum_{i=1}^{N} w_t^{(i)} \delta_{x_t^{(i)}}(x_t)$$

Where:
- $\{x_t^{(i)}\}_{i=1}^N$ = particles (samples)
- $\{w_t^{(i)}\}_{i=1}^N$ = normalized importance weights
- $\delta_{x}$ = Dirac delta function

### 2.2 Recursive Bayesian Filtering

**Prediction Step:**
$$p(x_t | y_{1:t-1}) = \int p(x_t | x_{t-1}) p(x_{t-1} | y_{1:t-1}) dx_{t-1}$$

**Update Step:**
$$p(x_t | y_{1:t}) = \frac{p(y_t | x_t) p(x_t | y_{1:t-1})}{p(y_t | y_{1:t-1})}$$

### 2.3 Importance Sampling

Since we can't sample directly from the posterior, we use a **proposal distribution** $q(x_t | x_{t-1}, y_t)$:

$$w_t^{(i)} \propto w_{t-1}^{(i)} \frac{p(y_t | x_t^{(i)}) p(x_t^{(i)} | x_{t-1}^{(i)})}{q(x_t^{(i)} | x_{t-1}^{(i)}, y_t)}$$

In [ ]:
# Visualize Importance Sampling Concept

def visualize_importance_sampling():
    """
    Demonstrate importance sampling for a non-Gaussian target.
    """
    # Target: Mixture of Gaussians (non-Gaussian)
    def target_pdf(x):
        return 0.3 * stats.norm.pdf(x, -2, 0.5) + 0.7 * stats.norm.pdf(x, 2, 1)
    
    # Proposal: Simple Gaussian
    proposal_mean, proposal_std = 0, 2.5
    
    # Draw samples from proposal
    N = 1000
    samples = np.random.normal(proposal_mean, proposal_std, N)
    
    # Compute importance weights
    target_vals = target_pdf(samples)
    proposal_vals = stats.norm.pdf(samples, proposal_mean, proposal_std)
    weights = target_vals / proposal_vals
    weights_normalized = weights / weights.sum()
    
    # Effective Sample Size
    ESS = 1 / np.sum(weights_normalized**2)
    
    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    x_range = np.linspace(-6, 6, 500)
    
    # Plot 1: Target and Proposal
    axes[0].plot(x_range, target_pdf(x_range), 'b-', lw=2, label='Target p(x)')
    axes[0].plot(x_range, stats.norm.pdf(x_range, proposal_mean, proposal_std), 
                 'r--', lw=2, label='Proposal q(x)')
    axes[0].fill_between(x_range, target_pdf(x_range), alpha=0.3)
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('Density')
    axes[0].set_title('Target vs Proposal Distribution')
    axes[0].legend()
    
    # Plot 2: Unweighted Samples
    axes[1].hist(samples, bins=50, density=True, alpha=0.6, color='red', 
                 label='Unweighted Samples')
    axes[1].plot(x_range, target_pdf(x_range), 'b-', lw=2, label='Target')
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('Density')
    axes[1].set_title('Unweighted Samples (from Proposal)')
    axes[1].legend()
    
    # Plot 3: Weighted Samples (Resampled)
    resampled_idx = np.random.choice(N, size=N, p=weights_normalized)
    resampled = samples[resampled_idx]
    axes[2].hist(resampled, bins=50, density=True, alpha=0.6, color='green',
                 label='Resampled')
    axes[2].plot(x_range, target_pdf(x_range), 'b-', lw=2, label='Target')
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('Density')
    axes[2].set_title(f'After Resampling (ESS={ESS:.0f}/{N})')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Importance Sampling Results:")
    print(f"   • Number of particles: {N}")
    print(f"   • Effective Sample Size (ESS): {ESS:.1f} ({100*ESS/N:.1f}%)")
    print(f"   • True mean: {0.3*(-2) + 0.7*2:.2f}")
    print(f"   • Weighted estimate: {np.sum(weights_normalized * samples):.2f}")

visualize_importance_sampling()

---

## 3. The Bootstrap Particle Filter <a name="3-bootstrap-pf"></a>

### 3.1 Algorithm (Sequential Importance Resampling - SIR)

The Bootstrap Filter uses the **state transition** as the proposal: $q(x_t | x_{t-1}, y_t) = p(x_t | x_{t-1})$

**Algorithm:**

```
1. INITIALIZE: Sample x_0^(i) ~ p(x_0), set w_0^(i) = 1/N

2. FOR t = 1, 2, ..., T:
   
   a. PREDICT: Sample x_t^(i) ~ p(x_t | x_{t-1}^(i))
   
   b. UPDATE: Compute weights w_t^(i) ∝ p(y_t | x_t^(i))
   
   c. NORMALIZE: w_t^(i) = w_t^(i) / Σ w_t^(j)
   
   d. RESAMPLE: If ESS < threshold, resample particles
   
   e. ESTIMATE: E[x_t] ≈ Σ w_t^(i) * x_t^(i)
```

### 3.2 Effective Sample Size (ESS)

$$\text{ESS} = \frac{1}{\sum_{i=1}^N (w_t^{(i)})^2}$$

- ESS ≈ N: All weights similar (good)
- ESS ≈ 1: One particle dominates (particle degeneracy - bad!)

In [ ]:
class BootstrapParticleFilter:
    """
    Bootstrap Particle Filter (SIR) implementation.
    
    Features:
    - Flexible state transition and observation models
    - Multiple resampling strategies
    - ESS monitoring and adaptive resampling
    """
    
    def __init__(self, n_particles, state_dim=1, resample_threshold=0.5,
                 resample_method='systematic'):
        """
        Initialize the particle filter.
        
        Parameters:
        -----------
        n_particles : int
            Number of particles
        state_dim : int
            Dimension of state space
        resample_threshold : float
            Resample when ESS/N < threshold
        resample_method : str
            'multinomial', 'systematic', 'stratified', 'residual'
        """
        self.N = n_particles
        self.state_dim = state_dim
        self.resample_threshold = resample_threshold
        self.resample_method = resample_method
        
        # Storage
        self.particles = None
        self.weights = None
        self.ess_history = []
        self.resample_times = []
        
    def initialize(self, prior_sampler):
        """
        Initialize particles from prior distribution.
        
        Parameters:
        -----------
        prior_sampler : callable
            Function that returns N samples from prior
        """
        self.particles = prior_sampler(self.N)
        self.weights = np.ones(self.N) / self.N
        self.ess_history = [self.N]
        self.resample_times = []
        
    def predict(self, transition_sampler):
        """
        Prediction step: propagate particles through state transition.
        
        Parameters:
        -----------
        transition_sampler : callable
            Function(particles) -> new_particles
        """
        self.particles = transition_sampler(self.particles)
        
    def update(self, observation, likelihood_func, t=None):
        """
        Update step: reweight particles based on observation likelihood.
        
        Parameters:
        -----------
        observation : float or array
            Current observation y_t
        likelihood_func : callable
            Function(particles, observation) -> likelihoods
        t : int, optional
            Current time step (for tracking)
        """
        # Compute likelihoods
        likelihoods = likelihood_func(self.particles, observation)
        
        # Update weights (in log space for numerical stability)
        log_weights = np.log(self.weights + 1e-300) + np.log(likelihoods + 1e-300)
        
        # Normalize
        log_weights -= logsumexp(log_weights)
        self.weights = np.exp(log_weights)
        
        # Compute ESS
        ess = self.compute_ess()
        self.ess_history.append(ess)
        
        # Adaptive resampling
        if ess < self.resample_threshold * self.N:
            self.resample()
            if t is not None:
                self.resample_times.append(t)
                
    def compute_ess(self):
        """Compute Effective Sample Size."""
        return 1.0 / np.sum(self.weights**2)
    
    def resample(self):
        """Resample particles according to their weights."""
        if self.resample_method == 'multinomial':
            indices = self._multinomial_resample()
        elif self.resample_method == 'systematic':
            indices = self._systematic_resample()
        elif self.resample_method == 'stratified':
            indices = self._stratified_resample()
        elif self.resample_method == 'residual':
            indices = self._residual_resample()
        else:
            raise ValueError(f"Unknown resampling method: {self.resample_method}")
            
        self.particles = self.particles[indices]
        self.weights = np.ones(self.N) / self.N
        
    def _multinomial_resample(self):
        """Standard multinomial resampling."""
        return np.random.choice(self.N, size=self.N, p=self.weights)
    
    def _systematic_resample(self):
        """Systematic resampling (lower variance)."""
        positions = (np.arange(self.N) + np.random.uniform()) / self.N
        cumsum = np.cumsum(self.weights)
        indices = np.searchsorted(cumsum, positions)
        return np.clip(indices, 0, self.N - 1)
    
    def _stratified_resample(self):
        """Stratified resampling."""
        positions = (np.arange(self.N) + np.random.uniform(size=self.N)) / self.N
        cumsum = np.cumsum(self.weights)
        indices = np.searchsorted(cumsum, positions)
        return np.clip(indices, 0, self.N - 1)
    
    def _residual_resample(self):
        """Residual resampling."""
        # Deterministic part
        num_copies = np.floor(self.N * self.weights).astype(int)
        indices = np.repeat(np.arange(self.N), num_copies)
        
        # Stochastic part
        residuals = self.N * self.weights - num_copies
        residuals /= residuals.sum()
        remaining = self.N - len(indices)
        additional = np.random.choice(self.N, size=remaining, p=residuals)
        
        return np.concatenate([indices, additional])
    
    def estimate_mean(self):
        """Compute weighted mean of particles."""
        return np.sum(self.weights[:, np.newaxis] * self.particles, axis=0)
    
    def estimate_variance(self):
        """Compute weighted variance of particles."""
        mean = self.estimate_mean()
        return np.sum(self.weights[:, np.newaxis] * (self.particles - mean)**2, axis=0)
    
    def estimate_quantiles(self, quantiles=[0.05, 0.5, 0.95]):
        """Compute weighted quantiles."""
        sorted_idx = np.argsort(self.particles.flatten())
        sorted_particles = self.particles.flatten()[sorted_idx]
        sorted_weights = self.weights[sorted_idx]
        cumsum = np.cumsum(sorted_weights)
        
        result = []
        for q in quantiles:
            idx = np.searchsorted(cumsum, q)
            idx = min(idx, len(sorted_particles) - 1)
            result.append(sorted_particles[idx])
        return result

print("✅ BootstrapParticleFilter class defined!")

In [ ]:
# Test the Bootstrap Particle Filter on the nonlinear system

def run_particle_filter_demo():
    """
    Apply Bootstrap PF to the same nonlinear system where Kalman failed.
    """
    # Generate data
    T = 100
    Q = 10  # Process noise variance
    R = 1   # Observation noise variance
    
    x_true = np.zeros(T)
    y = np.zeros(T)
    x_true[0] = 0.1
    
    for t in range(1, T):
        x_true[t] = (0.5 * x_true[t-1] + 
                     25 * x_true[t-1] / (1 + x_true[t-1]**2) + 
                     8 * np.cos(1.2 * t) + 
                     np.sqrt(Q) * np.random.randn())
    
    y = x_true**2 / 20 + np.sqrt(R) * np.random.randn(T)
    
    # Define state-space model functions
    def prior_sampler(N):
        return np.random.normal(0, 1, (N, 1))
    
    def transition_sampler(particles, t):
        """Nonlinear state transition."""
        x = particles.flatten()
        x_new = (0.5 * x + 
                 25 * x / (1 + x**2) + 
                 8 * np.cos(1.2 * t) + 
                 np.sqrt(Q) * np.random.randn(len(x)))
        return x_new.reshape(-1, 1)
    
    def likelihood(particles, observation):
        """Observation likelihood p(y|x) = N(y; x^2/20, R)."""
        x = particles.flatten()
        pred_obs = x**2 / 20
        return stats.norm.pdf(observation, pred_obs, np.sqrt(R))
    
    # Run particle filter
    n_particles = 500
    pf = BootstrapParticleFilter(n_particles, state_dim=1, 
                                  resample_threshold=0.5,
                                  resample_method='systematic')
    pf.initialize(prior_sampler)
    
    # Storage for estimates
    x_pf_mean = np.zeros(T)
    x_pf_std = np.zeros(T)
    x_pf_q05 = np.zeros(T)
    x_pf_q95 = np.zeros(T)
    
    x_pf_mean[0] = pf.estimate_mean()[0]
    x_pf_std[0] = np.sqrt(pf.estimate_variance()[0])
    
    for t in range(1, T):
        # Predict
        pf.predict(lambda p: transition_sampler(p, t))
        
        # Update
        pf.update(y[t], likelihood, t=t)
        
        # Store estimates
        x_pf_mean[t] = pf.estimate_mean()[0]
        x_pf_std[t] = np.sqrt(pf.estimate_variance()[0])
        q05, _, q95 = pf.estimate_quantiles([0.05, 0.5, 0.95])
        x_pf_q05[t] = q05
        x_pf_q95[t] = q95
    
    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: State estimation
    axes[0, 0].plot(x_true, 'b-', lw=2, label='True State', alpha=0.8)
    axes[0, 0].plot(x_pf_mean, 'r-', lw=2, label='PF Estimate', alpha=0.8)
    axes[0, 0].fill_between(range(T), x_pf_q05, x_pf_q95, 
                            alpha=0.3, color='red', label='90% CI')
    axes[0, 0].set_xlabel('Time')
    axes[0, 0].set_ylabel('State')
    axes[0, 0].set_title('Particle Filter State Estimation')
    axes[0, 0].legend()
    
    # Plot 2: Estimation error
    error = x_true - x_pf_mean
    axes[0, 1].plot(error, 'g-', lw=1, alpha=0.8)
    axes[0, 1].axhline(0, color='k', linestyle='--')
    axes[0, 1].fill_between(range(T), -2*x_pf_std, 2*x_pf_std, 
                            alpha=0.2, color='blue', label='±2σ')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('Error')
    axes[0, 1].set_title(f'Estimation Error (RMSE={np.sqrt(np.mean(error**2)):.2f})')
    axes[0, 1].legend()
    
    # Plot 3: ESS over time
    axes[1, 0].plot(pf.ess_history, 'b-', lw=1)
    axes[1, 0].axhline(n_particles, color='g', linestyle='--', label='N particles')
    axes[1, 0].axhline(0.5 * n_particles, color='r', linestyle='--', label='Resample threshold')
    for rt in pf.resample_times:
        axes[1, 0].axvline(rt, color='orange', alpha=0.3, lw=0.5)
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('ESS')
    axes[1, 0].set_title(f'Effective Sample Size (Resampled {len(pf.resample_times)} times)')
    axes[1, 0].legend()
    
    # Plot 4: Final particle distribution
    axes[1, 1].hist(pf.particles.flatten(), bins=50, density=True, alpha=0.6, color='blue')
    axes[1, 1].axvline(x_true[-1], color='r', lw=2, label=f'True: {x_true[-1]:.2f}')
    axes[1, 1].axvline(x_pf_mean[-1], color='g', lw=2, linestyle='--', 
                       label=f'Est: {x_pf_mean[-1]:.2f}')
    axes[1, 1].set_xlabel('State')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Final Particle Distribution')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Particle Filter Performance:")
    print(f"   • RMSE: {np.sqrt(np.mean(error**2)):.3f}")
    print(f"   • Mean ESS: {np.mean(pf.ess_history):.1f} / {n_particles}")
    print(f"   • Resample events: {len(pf.resample_times)}")
    
    return pf

pf = run_particle_filter_demo()

---

## 4. Resampling Strategies <a name="4-resampling"></a>

### 4.1 Why Resampling?

**Problem: Weight Degeneracy**
- Over time, weights become concentrated on few particles
- Most particles have negligible weights
- ESS → 1 (only one effective particle)

**Solution: Resampling**
- Duplicate high-weight particles
- Remove low-weight particles
- Reset all weights to 1/N

### 4.2 Comparison of Methods

| Method | Variance | Complexity | Notes |
|--------|----------|------------|-------|
| Multinomial | High | O(N log N) | Simple but high variance |
| Systematic | Low | O(N) | **Preferred in practice** |
| Stratified | Low | O(N) | Good balance |
| Residual | Medium | O(N) | Hybrid approach |

In [ ]:
# Compare resampling methods

def compare_resampling_methods():
    """
    Compare variance and performance of different resampling strategies.
    """
    # Create a skewed weight distribution
    N = 100
    np.random.seed(42)
    
    # Exponential weights (highly skewed)
    raw_weights = np.exp(np.random.randn(N) * 2)
    weights = raw_weights / raw_weights.sum()
    
    # Original particles
    particles = np.arange(N)
    
    # Resampling functions
    def multinomial_resample(w):
        return np.random.choice(len(w), size=len(w), p=w)
    
    def systematic_resample(w):
        N = len(w)
        positions = (np.arange(N) + np.random.uniform()) / N
        cumsum = np.cumsum(w)
        return np.clip(np.searchsorted(cumsum, positions), 0, N-1)
    
    def stratified_resample(w):
        N = len(w)
        positions = (np.arange(N) + np.random.uniform(size=N)) / N
        cumsum = np.cumsum(w)
        return np.clip(np.searchsorted(cumsum, positions), 0, N-1)
    
    # Run multiple trials
    n_trials = 1000
    methods = {
        'Multinomial': multinomial_resample,
        'Systematic': systematic_resample,
        'Stratified': stratified_resample
    }
    
    # True expectation
    true_mean = np.sum(weights * particles)
    
    results = {name: [] for name in methods}
    
    for _ in range(n_trials):
        for name, func in methods.items():
            idx = func(weights)
            estimate = np.mean(particles[idx])
            results[name].append(estimate)
    
    # Plot results
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Plot 1: Original weight distribution
    axes[0].bar(range(N), np.sort(weights)[::-1], alpha=0.7)
    axes[0].set_xlabel('Particle Index (sorted)')
    axes[0].set_ylabel('Weight')
    axes[0].set_title(f'Weight Distribution (ESS={1/np.sum(weights**2):.1f})')
    
    # Plot 2: Histogram of estimates
    colors = ['blue', 'green', 'orange']
    for (name, estimates), color in zip(results.items(), colors):
        axes[1].hist(estimates, bins=30, alpha=0.5, label=name, color=color, density=True)
    axes[1].axvline(true_mean, color='red', lw=2, linestyle='--', label='True Mean')
    axes[1].set_xlabel('Estimate')
    axes[1].set_ylabel('Density')
    axes[1].set_title('Distribution of Mean Estimates')
    axes[1].legend()
    
    # Plot 3: Variance comparison
    variances = [np.var(results[name]) for name in methods]
    bars = axes[2].bar(methods.keys(), variances, color=colors, alpha=0.7)
    axes[2].set_ylabel('Variance of Estimate')
    axes[2].set_title('Resampling Variance Comparison')
    
    # Add variance values on bars
    for bar, var in zip(bars, variances):
        axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                     f'{var:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Resampling Method Comparison:")
    print(f"   True mean: {true_mean:.2f}")
    print(f"   {'Method':<15} {'Mean':<10} {'Std':<10} {'Bias':<10}")
    print("   " + "-" * 45)
    for name in methods:
        mean_est = np.mean(results[name])
        std_est = np.std(results[name])
        bias = mean_est - true_mean
        print(f"   {name:<15} {mean_est:<10.3f} {std_est:<10.3f} {bias:<10.4f}")

compare_resampling_methods()

---

## 5. Financial Applications <a name="5-finance-applications"></a>

### 5.1 Stochastic Volatility Model

One of the most important applications in finance!

**Model:**
$$\begin{align}
r_t &= \exp(h_t/2) \epsilon_t \quad &\epsilon_t \sim N(0,1) \\
h_t &= \mu + \phi(h_{t-1} - \mu) + \sigma_h \eta_t \quad &\eta_t \sim N(0,1)
\end{align}$$

Where:
- $r_t$ = log returns (observed)
- $h_t$ = log volatility (hidden state)
- $\mu$ = long-run mean log volatility
- $\phi$ = persistence parameter (close to 1)
- $\sigma_h$ = volatility of volatility

**Why Particle Filter?**
- Observation equation $r_t = \exp(h_t/2) \epsilon_t$ is **nonlinear** in $h_t$
- The likelihood $p(r_t | h_t)$ involves a nonlinear transformation

In [ ]:
class StochasticVolatilityPF:
    """
    Particle Filter for Stochastic Volatility Model.
    
    State: h_t (log volatility)
    Observation: r_t (returns)
    """
    
    def __init__(self, mu, phi, sigma_h, n_particles=1000):
        """
        Parameters:
        -----------
        mu : float
            Long-run mean of log volatility
        phi : float
            Persistence parameter (0 < phi < 1)
        sigma_h : float
            Volatility of volatility
        n_particles : int
            Number of particles
        """
        self.mu = mu
        self.phi = phi
        self.sigma_h = sigma_h
        self.N = n_particles
        
        # Unconditional distribution
        self.unconditional_var = sigma_h**2 / (1 - phi**2)
        
    def simulate(self, T):
        """
        Simulate from the SV model.
        """
        h = np.zeros(T)
        r = np.zeros(T)
        
        # Initialize from unconditional distribution
        h[0] = self.mu + np.sqrt(self.unconditional_var) * np.random.randn()
        r[0] = np.exp(h[0]/2) * np.random.randn()
        
        for t in range(1, T):
            h[t] = self.mu + self.phi * (h[t-1] - self.mu) + self.sigma_h * np.random.randn()
            r[t] = np.exp(h[t]/2) * np.random.randn()
            
        return r, h
    
    def filter(self, returns):
        """
        Run particle filter on observed returns.
        
        Returns:
        --------
        h_mean : array
            Filtered mean of log volatility
        h_std : array
            Filtered std of log volatility
        vol_mean : array
            Filtered mean of volatility (exp(h/2))
        """
        T = len(returns)
        
        # Initialize particles from unconditional distribution
        particles = self.mu + np.sqrt(self.unconditional_var) * np.random.randn(self.N)
        weights = np.ones(self.N) / self.N
        
        # Storage
        h_mean = np.zeros(T)
        h_std = np.zeros(T)
        h_q05 = np.zeros(T)
        h_q95 = np.zeros(T)
        ess_history = []
        
        for t in range(T):
            if t > 0:
                # Prediction step
                particles = (self.mu + self.phi * (particles - self.mu) + 
                            self.sigma_h * np.random.randn(self.N))
            
            # Update step
            # Likelihood: p(r_t | h_t) = N(r_t; 0, exp(h_t))
            vol = np.exp(particles / 2)
            log_likelihood = -0.5 * np.log(2 * np.pi) - particles/2 - 0.5 * (returns[t]**2) / np.exp(particles)
            
            # Update weights (log space)
            log_weights = np.log(weights + 1e-300) + log_likelihood
            log_weights -= logsumexp(log_weights)
            weights = np.exp(log_weights)
            
            # Store estimates
            h_mean[t] = np.sum(weights * particles)
            h_std[t] = np.sqrt(np.sum(weights * (particles - h_mean[t])**2))
            
            # Quantiles
            sorted_idx = np.argsort(particles)
            cumsum = np.cumsum(weights[sorted_idx])
            h_q05[t] = particles[sorted_idx][np.searchsorted(cumsum, 0.05)]
            h_q95[t] = particles[sorted_idx][min(np.searchsorted(cumsum, 0.95), self.N-1)]
            
            # ESS
            ess = 1.0 / np.sum(weights**2)
            ess_history.append(ess)
            
            # Resample if ESS too low
            if ess < self.N / 2:
                indices = np.random.choice(self.N, size=self.N, p=weights)
                particles = particles[indices]
                weights = np.ones(self.N) / self.N
        
        vol_mean = np.exp(h_mean / 2)
        
        return {
            'h_mean': h_mean,
            'h_std': h_std,
            'h_q05': h_q05,
            'h_q95': h_q95,
            'vol_mean': vol_mean,
            'ess': np.array(ess_history)
        }

# Run SV model example
def run_sv_example():
    # Parameters (typical for daily returns)
    mu = -0.5      # Long-run mean log vol (corresponds to ~22% annual vol)
    phi = 0.98     # High persistence
    sigma_h = 0.15 # Vol of vol
    
    # Simulate data
    T = 500
    sv_model = StochasticVolatilityPF(mu, phi, sigma_h, n_particles=2000)
    returns, h_true = sv_model.simulate(T)
    
    # Run particle filter
    results = sv_model.filter(returns)
    
    # Plot results
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # Plot 1: Returns
    axes[0].plot(returns, 'b-', lw=0.5, alpha=0.7)
    axes[0].set_ylabel('Returns')
    axes[0].set_title('Simulated Returns from Stochastic Volatility Model')
    axes[0].axhline(0, color='k', linestyle='--', alpha=0.3)
    
    # Plot 2: Log volatility
    axes[1].plot(h_true, 'b-', lw=1.5, label='True h_t', alpha=0.8)
    axes[1].plot(results['h_mean'], 'r-', lw=1.5, label='Filtered h_t', alpha=0.8)
    axes[1].fill_between(range(T), results['h_q05'], results['h_q95'],
                         alpha=0.3, color='red', label='90% CI')
    axes[1].set_ylabel('Log Volatility h_t')
    axes[1].set_title('Particle Filter: Log Volatility Estimation')
    axes[1].legend()
    
    # Plot 3: Volatility
    vol_true = np.exp(h_true / 2)
    abs_returns = np.abs(returns)
    
    axes[2].plot(vol_true, 'b-', lw=1.5, label='True σ_t', alpha=0.8)
    axes[2].plot(results['vol_mean'], 'r-', lw=1.5, label='Filtered σ_t', alpha=0.8)
    axes[2].plot(abs_returns, 'g.', alpha=0.2, markersize=2, label='|r_t|')
    axes[2].set_xlabel('Time')
    axes[2].set_ylabel('Volatility')
    axes[2].set_title('Particle Filter: Volatility Estimation')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Metrics
    rmse_h = np.sqrt(np.mean((h_true - results['h_mean'])**2))
    rmse_vol = np.sqrt(np.mean((vol_true - results['vol_mean'])**2))
    corr_vol = np.corrcoef(vol_true, results['vol_mean'])[0, 1]
    
    print(f"\n📊 Stochastic Volatility Filter Performance:")
    print(f"   • Log-vol RMSE: {rmse_h:.4f}")
    print(f"   • Volatility RMSE: {rmse_vol:.4f}")
    print(f"   • Volatility Correlation: {corr_vol:.4f}")
    print(f"   • Mean ESS: {np.mean(results['ess']):.0f} / {sv_model.N}")
    
    return returns, h_true, results

returns, h_true, sv_results = run_sv_example()

### 5.2 Regime-Switching Model

**Hamilton Regime-Switching Model:**

$$r_t = \mu_{s_t} + \sigma_{s_t} \epsilon_t$$

Where $s_t \in \{0, 1\}$ follows a Markov chain with transition matrix:

$$P = \begin{pmatrix} p_{00} & p_{01} \\ p_{10} & p_{11} \end{pmatrix}$$

**Why Particle Filter?**
- Discrete state space (not Gaussian!)
- Mixture distribution for observations

In [ ]:
class RegimeSwitchingPF:
    """
    Particle Filter for 2-state Regime-Switching Model.
    
    State: s_t ∈ {0, 1} (regime)
    Observation: r_t (returns)
    """
    
    def __init__(self, mu, sigma, P, n_particles=1000):
        """
        Parameters:
        -----------
        mu : array
            Mean returns in each regime [mu_0, mu_1]
        sigma : array
            Volatility in each regime [sigma_0, sigma_1]
        P : array (2x2)
            Transition probability matrix
        n_particles : int
            Number of particles
        """
        self.mu = np.array(mu)
        self.sigma = np.array(sigma)
        self.P = np.array(P)
        self.N = n_particles
        
        # Stationary distribution
        self.pi = self._compute_stationary()
        
    def _compute_stationary(self):
        """Compute stationary distribution of Markov chain."""
        # pi * P = pi, sum(pi) = 1
        # pi_0 * P[0,0] + pi_1 * P[1,0] = pi_0
        # pi_0 + pi_1 = 1
        p01 = self.P[0, 1]
        p10 = self.P[1, 0]
        pi_0 = p10 / (p01 + p10)
        return np.array([pi_0, 1 - pi_0])
    
    def simulate(self, T):
        """
        Simulate from the regime-switching model.
        """
        s = np.zeros(T, dtype=int)
        r = np.zeros(T)
        
        # Initialize from stationary distribution
        s[0] = np.random.choice([0, 1], p=self.pi)
        r[0] = self.mu[s[0]] + self.sigma[s[0]] * np.random.randn()
        
        for t in range(1, T):
            # Transition
            s[t] = np.random.choice([0, 1], p=self.P[s[t-1]])
            # Observation
            r[t] = self.mu[s[t]] + self.sigma[s[t]] * np.random.randn()
            
        return r, s
    
    def filter(self, returns):
        """
        Run particle filter.
        
        Returns filtered probability of being in regime 1.
        """
        T = len(returns)
        
        # Initialize particles
        particles = np.random.choice([0, 1], size=self.N, p=self.pi)
        weights = np.ones(self.N) / self.N
        
        # Storage
        prob_regime1 = np.zeros(T)
        ess_history = []
        
        for t in range(T):
            if t > 0:
                # Prediction: propagate through Markov transition
                new_particles = np.zeros(self.N, dtype=int)
                for i in range(self.N):
                    new_particles[i] = np.random.choice([0, 1], p=self.P[particles[i]])
                particles = new_particles
            
            # Update: compute likelihood
            likelihood = np.zeros(self.N)
            for i in range(self.N):
                likelihood[i] = stats.norm.pdf(returns[t], 
                                                self.mu[particles[i]], 
                                                self.sigma[particles[i]])
            
            # Update weights
            log_weights = np.log(weights + 1e-300) + np.log(likelihood + 1e-300)
            log_weights -= logsumexp(log_weights)
            weights = np.exp(log_weights)
            
            # Store regime probability
            prob_regime1[t] = np.sum(weights * particles)
            
            # ESS
            ess = 1.0 / np.sum(weights**2)
            ess_history.append(ess)
            
            # Resample
            if ess < self.N / 2:
                indices = np.random.choice(self.N, size=self.N, p=weights)
                particles = particles[indices]
                weights = np.ones(self.N) / self.N
        
        return prob_regime1, np.array(ess_history)

# Run regime-switching example
def run_regime_switching_example():
    # Parameters: Bull and Bear market
    mu = [0.001, -0.002]     # Bull: +0.1%, Bear: -0.2% daily
    sigma = [0.01, 0.025]    # Bull: 1%, Bear: 2.5% daily vol
    P = [[0.98, 0.02],       # 2% chance of switching from Bull to Bear
         [0.05, 0.95]]       # 5% chance of switching from Bear to Bull
    
    T = 500
    rs_model = RegimeSwitchingPF(mu, sigma, P, n_particles=1000)
    returns, regimes = rs_model.simulate(T)
    
    # Run particle filter
    prob_bear, ess = rs_model.filter(returns)
    
    # Plot
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Plot 1: Returns colored by regime
    colors = ['green' if s == 0 else 'red' for s in regimes]
    axes[0].bar(range(T), returns, color=colors, alpha=0.7, width=1.0)
    axes[0].set_ylabel('Returns')
    axes[0].set_title('Returns (Green=Bull, Red=Bear)')
    
    # Plot 2: True regime vs filtered probability
    axes[1].fill_between(range(T), 0, regimes, alpha=0.3, color='red', 
                         label='True Bear Regime', step='mid')
    axes[1].plot(prob_bear, 'b-', lw=1.5, label='P(Bear|data)', alpha=0.8)
    axes[1].set_ylabel('Probability')
    axes[1].set_title('Regime Detection: True vs Filtered')
    axes[1].legend()
    axes[1].set_ylim(-0.1, 1.1)
    
    # Plot 3: Cumulative returns
    cumret = np.cumsum(returns)
    axes[2].plot(cumret, 'b-', lw=1.5)
    
    # Shade bear regimes
    in_bear = False
    bear_start = 0
    for t in range(T):
        if regimes[t] == 1 and not in_bear:
            bear_start = t
            in_bear = True
        elif regimes[t] == 0 and in_bear:
            axes[2].axvspan(bear_start, t, alpha=0.2, color='red')
            in_bear = False
    if in_bear:
        axes[2].axvspan(bear_start, T, alpha=0.2, color='red')
        
    axes[2].set_xlabel('Time')
    axes[2].set_ylabel('Cumulative Return')
    axes[2].set_title('Cumulative Returns (Red shading = Bear regime)')
    
    plt.tight_layout()
    plt.show()
    
    # Performance metrics
    regime_pred = (prob_bear > 0.5).astype(int)
    accuracy = np.mean(regime_pred == regimes)
    
    print(f"\n📊 Regime-Switching Filter Performance:")
    print(f"   • Classification Accuracy: {accuracy:.1%}")
    print(f"   • Mean ESS: {np.mean(ess):.0f} / {rs_model.N}")
    print(f"   • Stationary prob (Bear): {rs_model.pi[1]:.1%}")
    print(f"   • Empirical Bear freq: {np.mean(regimes):.1%}")
    
    return returns, regimes, prob_bear

rs_returns, rs_regimes, rs_prob = run_regime_switching_example()

### 5.3 Application to Real Financial Data

Let's apply the Stochastic Volatility particle filter to real market data!

In [ ]:
def apply_sv_to_real_data():
    """
    Apply Stochastic Volatility PF to simulated "real" market data.
    (In practice, you'd use yfinance or similar)
    """
    np.random.seed(42)
    
    # Simulate realistic market data with fat tails and volatility clustering
    T = 750  # ~3 years of daily data
    
    # Generate returns with realistic features
    # Use a more complex model to simulate
    mu_sv = -0.3
    phi_sv = 0.985
    sigma_h_sv = 0.12
    
    h = np.zeros(T)
    h[0] = mu_sv
    
    for t in range(1, T):
        # Add some jumps to volatility
        jump = 0.5 * (np.random.random() < 0.02)  # 2% chance of vol spike
        h[t] = mu_sv + phi_sv * (h[t-1] - mu_sv) + sigma_h_sv * np.random.randn() + jump
    
    # Generate returns with Student-t innovations (fatter tails)
    df = 5  # degrees of freedom
    returns = np.exp(h/2) * np.random.standard_t(df, T) / np.sqrt(df/(df-2))
    
    # Convert to price
    prices = 100 * np.exp(np.cumsum(returns))
    
    # Create date index
    dates = pd.date_range(start='2023-01-01', periods=T, freq='B')
    
    # Run particle filter (with parameters we'd typically estimate)
    sv_filter = StochasticVolatilityPF(
        mu=-0.4,       # Slightly misspecified
        phi=0.98,
        sigma_h=0.15,
        n_particles=3000
    )
    
    results = sv_filter.filter(returns)
    
    # Compare with realized volatility (20-day rolling)
    realized_vol = pd.Series(returns).rolling(20).std() * np.sqrt(252)
    filtered_vol_annual = results['vol_mean'] * np.sqrt(252)
    
    # Plot
    fig, axes = plt.subplots(4, 1, figsize=(14, 14))
    
    # Plot 1: Price
    axes[0].plot(dates, prices, 'b-', lw=1)
    axes[0].set_ylabel('Price')
    axes[0].set_title('Simulated Market Data with Stochastic Volatility')
    
    # Plot 2: Returns
    axes[1].plot(dates, returns * 100, 'b-', lw=0.5, alpha=0.7)
    axes[1].axhline(0, color='k', linestyle='--', alpha=0.3)
    axes[1].set_ylabel('Returns (%)')
    axes[1].set_title('Daily Returns')
    
    # Plot 3: Volatility comparison
    axes[2].plot(dates, realized_vol, 'b-', lw=1, label='20-day Realized Vol', alpha=0.7)
    axes[2].plot(dates, filtered_vol_annual, 'r-', lw=1.5, label='PF Filtered Vol', alpha=0.8)
    axes[2].plot(dates, np.exp(h/2) * np.sqrt(252), 'g--', lw=1, label='True Vol', alpha=0.7)
    axes[2].set_ylabel('Annualized Volatility')
    axes[2].set_title('Volatility: Particle Filter vs Realized')
    axes[2].legend()
    
    # Plot 4: ESS
    axes[3].plot(dates, results['ess'], 'b-', lw=0.5)
    axes[3].axhline(sv_filter.N * 0.5, color='r', linestyle='--', label='Resample threshold')
    axes[3].set_xlabel('Date')
    axes[3].set_ylabel('ESS')
    axes[3].set_title('Effective Sample Size')
    axes[3].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Correlation analysis
    valid_idx = ~np.isnan(realized_vol)
    corr = np.corrcoef(realized_vol[valid_idx], filtered_vol_annual[valid_idx])[0, 1]
    
    print(f"\n📊 Real Data Analysis Results:")
    print(f"   • Correlation (PF vol vs Realized vol): {corr:.3f}")
    print(f"   • Mean filtered vol: {np.mean(filtered_vol_annual)*100:.1f}%")
    print(f"   • Mean realized vol: {np.nanmean(realized_vol)*100:.1f}%")
    print(f"   • Vol of vol (PF): {np.std(filtered_vol_annual)*100:.2f}%")
    
    return results

real_results = apply_sv_to_real_data()

---

## 6. Advanced Topics <a name="6-advanced"></a>

### 6.1 Auxiliary Particle Filter (APF)

**Improvement over Bootstrap PF:**
- Look ahead using the observation to guide resampling
- Better proposal that considers $y_t$

**Key Idea:** First resample based on predicted observation likelihood, then propagate.

### 6.2 Rao-Blackwellized Particle Filter

**For models with linear sub-structure:**
- Marginalize out the linear part analytically (Kalman filter)
- Use particles only for the nonlinear part
- Reduces variance significantly

### 6.3 Particle MCMC

**For parameter estimation:**
- Use particle filter within MCMC
- Particle Marginal Metropolis-Hastings (PMMH)
- Achieves exact Bayesian inference

In [ ]:
class AuxiliaryParticleFilter:
    """
    Auxiliary Particle Filter for Stochastic Volatility.
    
    Improves upon Bootstrap PF by using a look-ahead step.
    """
    
    def __init__(self, mu, phi, sigma_h, n_particles=1000):
        self.mu = mu
        self.phi = phi
        self.sigma_h = sigma_h
        self.N = n_particles
        self.unconditional_var = sigma_h**2 / (1 - phi**2)
        
    def filter(self, returns):
        """
        Run Auxiliary Particle Filter.
        """
        T = len(returns)
        
        # Initialize
        particles = self.mu + np.sqrt(self.unconditional_var) * np.random.randn(self.N)
        weights = np.ones(self.N) / self.N
        
        h_mean = np.zeros(T)
        ess_history = []
        
        for t in range(T):
            if t > 0:
                # APF Step 1: Compute auxiliary weights using predicted mean
                h_pred_mean = self.mu + self.phi * (particles - self.mu)
                
                # Likelihood at predicted mean
                aux_likelihood = stats.norm.pdf(returns[t], 0, np.exp(h_pred_mean/2))
                aux_weights = weights * aux_likelihood
                aux_weights /= aux_weights.sum()
                
                # First-stage resampling based on auxiliary weights
                indices = np.random.choice(self.N, size=self.N, p=aux_weights)
                particles = particles[indices]
                
                # APF Step 2: Propagate particles
                particles = (self.mu + self.phi * (particles - self.mu) + 
                            self.sigma_h * np.random.randn(self.N))
                
                # APF Step 3: Compute corrected weights
                likelihood = stats.norm.pdf(returns[t], 0, np.exp(particles/2))
                h_pred_resampled = self.mu + self.phi * (particles - self.mu)
                aux_likelihood_resampled = stats.norm.pdf(returns[t], 0, np.exp(h_pred_resampled/2))
                
                weights = likelihood / (aux_likelihood_resampled + 1e-300)
                weights /= weights.sum()
            else:
                # First observation
                likelihood = stats.norm.pdf(returns[t], 0, np.exp(particles/2))
                weights = weights * likelihood
                weights /= weights.sum()
            
            # Store estimates
            h_mean[t] = np.sum(weights * particles)
            ess = 1.0 / np.sum(weights**2)
            ess_history.append(ess)
            
            # Resample if needed
            if ess < self.N / 2:
                indices = np.random.choice(self.N, size=self.N, p=weights)
                particles = particles[indices]
                weights = np.ones(self.N) / self.N
        
        return {'h_mean': h_mean, 'vol_mean': np.exp(h_mean/2), 'ess': np.array(ess_history)}

# Compare Bootstrap PF vs Auxiliary PF
def compare_pf_methods():
    # Parameters
    mu, phi, sigma_h = -0.5, 0.98, 0.15
    T = 300
    N_particles = 500
    
    # Generate data
    sv_model = StochasticVolatilityPF(mu, phi, sigma_h, N_particles)
    returns, h_true = sv_model.simulate(T)
    
    # Run both filters multiple times
    n_runs = 10
    
    bpf_rmse = []
    apf_rmse = []
    bpf_ess = []
    apf_ess = []
    
    for _ in range(n_runs):
        # Bootstrap PF
        bpf = StochasticVolatilityPF(mu, phi, sigma_h, N_particles)
        bpf_results = bpf.filter(returns)
        bpf_rmse.append(np.sqrt(np.mean((h_true - bpf_results['h_mean'])**2)))
        bpf_ess.append(np.mean(bpf_results['ess']))
        
        # Auxiliary PF
        apf = AuxiliaryParticleFilter(mu, phi, sigma_h, N_particles)
        apf_results = apf.filter(returns)
        apf_rmse.append(np.sqrt(np.mean((h_true - apf_results['h_mean'])**2)))
        apf_ess.append(np.mean(apf_results['ess']))
    
    # Plot comparison
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # RMSE comparison
    methods = ['Bootstrap PF', 'Auxiliary PF']
    rmses = [np.mean(bpf_rmse), np.mean(apf_rmse)]
    rmse_stds = [np.std(bpf_rmse), np.std(apf_rmse)]
    
    axes[0].bar(methods, rmses, yerr=rmse_stds, capsize=5, color=['blue', 'green'], alpha=0.7)
    axes[0].set_ylabel('RMSE')
    axes[0].set_title('Log-Volatility RMSE Comparison')
    
    # ESS comparison
    ess_means = [np.mean(bpf_ess), np.mean(apf_ess)]
    ess_stds = [np.std(bpf_ess), np.std(apf_ess)]
    
    axes[1].bar(methods, ess_means, yerr=ess_stds, capsize=5, color=['blue', 'green'], alpha=0.7)
    axes[1].axhline(N_particles, color='r', linestyle='--', label=f'N={N_particles}')
    axes[1].set_ylabel('Mean ESS')
    axes[1].set_title('Effective Sample Size Comparison')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Method Comparison ({n_runs} runs, {N_particles} particles):")
    print(f"   {'Method':<15} {'RMSE':<15} {'Mean ESS':<15}")
    print("   " + "-" * 45)
    print(f"   {'Bootstrap PF':<15} {np.mean(bpf_rmse):.4f} ± {np.std(bpf_rmse):.4f}  {np.mean(bpf_ess):.1f} ± {np.std(bpf_ess):.1f}")
    print(f"   {'Auxiliary PF':<15} {np.mean(apf_rmse):.4f} ± {np.std(apf_rmse):.4f}  {np.mean(apf_ess):.1f} ± {np.std(apf_ess):.1f}")

compare_pf_methods()

---

## 7. Interview Questions & Exercises <a name="7-interview"></a>

### 💼 Common Interview Questions

**Q1: When would you use a particle filter instead of a Kalman filter?**

> Use particle filters when:
> - State transition is nonlinear
> - Observation model is nonlinear
> - Noise is non-Gaussian (fat tails, skewness)
> - State space is discrete (regime-switching)
> - Posterior is multimodal

**Q2: What is particle degeneracy and how do you address it?**

> Particle degeneracy occurs when few particles have most of the weight (ESS→1).
> Solutions:
> - Resampling (multinomial, systematic, stratified)
> - Better proposal distributions (auxiliary PF, optimal proposal)
> - More particles
> - Rao-Blackwellization

**Q3: Explain the tradeoff between number of particles and computational cost.**

> - Monte Carlo error ∝ 1/√N (need 4x particles for 2x accuracy)
> - Computational cost is O(N) per timestep
> - Memory cost is O(N)
> - In practice: 100-10,000 particles depending on problem complexity

**Q4: How would you use a particle filter for real-time trading?**

> Applications:
> - Real-time volatility estimation (stochastic vol model)
> - Regime detection for strategy switching
> - Tracking hidden market state (informed trading)
> - Online parameter learning
> 
> Considerations:
> - Latency requirements
> - Parallelization on GPU
> - Fixed-lag smoothing for better estimates

In [ ]:
# Exercise 1: Implement a particle filter for jump-diffusion model

def exercise_jump_diffusion_pf():
    """
    Exercise: Implement PF for Merton Jump-Diffusion.
    
    Model:
    dS = μS dt + σS dW + S dJ
    
    Where J is a compound Poisson process with:
    - Jump intensity λ
    - Jump size ~ N(μ_J, σ_J²)
    
    Hidden state: Number of jumps in each period
    Observation: Log returns
    """
    
    # Parameters
    mu = 0.0005      # Drift
    sigma = 0.01     # Diffusion volatility
    lam = 0.05       # Jump intensity (5% chance per day)
    mu_J = -0.02     # Mean jump size (negative = crashes)
    sigma_J = 0.03   # Jump size volatility
    
    T = 500
    N = 1000  # particles
    
    # Simulate data
    np.random.seed(42)
    n_jumps = np.random.poisson(lam, T)
    jump_sizes = np.array([np.sum(np.random.normal(mu_J, sigma_J, n)) if n > 0 else 0 
                           for n in n_jumps])
    returns = mu + sigma * np.random.randn(T) + jump_sizes
    
    # Particle filter
    # State: n_t (number of jumps)
    particles = np.zeros(N, dtype=int)  # Start with 0 jumps
    weights = np.ones(N) / N
    
    n_jumps_est = np.zeros(T)
    prob_jump = np.zeros(T)
    
    for t in range(T):
        # Prediction: sample number of jumps from Poisson
        particles = np.random.poisson(lam, N)
        
        # Update: compute likelihood
        # p(r_t | n_t) = N(r_t; μ + n_t*μ_J, σ² + n_t*σ_J²)
        likelihood = np.zeros(N)
        for i in range(N):
            mean_r = mu + particles[i] * mu_J
            var_r = sigma**2 + particles[i] * sigma_J**2
            likelihood[i] = stats.norm.pdf(returns[t], mean_r, np.sqrt(var_r))
        
        # Update weights
        weights = weights * likelihood
        weights /= weights.sum()
        
        # Estimates
        n_jumps_est[t] = np.sum(weights * particles)
        prob_jump[t] = np.sum(weights * (particles > 0))
        
        # Resample
        ess = 1.0 / np.sum(weights**2)
        if ess < N / 2:
            indices = np.random.choice(N, size=N, p=weights)
            particles = particles[indices]
            weights = np.ones(N) / N
    
    # Plot
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    axes[0].plot(returns * 100, 'b-', lw=0.5, alpha=0.7)
    for t in np.where(n_jumps > 0)[0]:
        axes[0].axvline(t, color='red', alpha=0.3, lw=1)
    axes[0].set_ylabel('Returns (%)')
    axes[0].set_title('Returns with Jump Events (red lines = actual jumps)')
    
    axes[1].plot(n_jumps, 'b-', lw=1, label='True # Jumps', alpha=0.8)
    axes[1].plot(n_jumps_est, 'r-', lw=1, label='Estimated # Jumps', alpha=0.8)
    axes[1].set_ylabel('Number of Jumps')
    axes[1].set_title('Jump Detection')
    axes[1].legend()
    
    axes[2].fill_between(range(T), 0, (n_jumps > 0).astype(float), 
                         alpha=0.3, color='blue', label='True Jump', step='mid')
    axes[2].plot(prob_jump, 'r-', lw=1, label='P(Jump|data)', alpha=0.8)
    axes[2].set_xlabel('Time')
    axes[2].set_ylabel('Probability')
    axes[2].set_title('Jump Probability Detection')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Metrics
    pred_jump = (prob_jump > 0.5).astype(int)
    true_jump = (n_jumps > 0).astype(int)
    
    tp = np.sum((pred_jump == 1) & (true_jump == 1))
    fp = np.sum((pred_jump == 1) & (true_jump == 0))
    fn = np.sum((pred_jump == 0) & (true_jump == 1))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"\n📊 Jump-Diffusion PF Results:")
    print(f"   • Total jumps: {np.sum(n_jumps > 0)}")
    print(f"   • Detected jumps: {np.sum(pred_jump)}")
    print(f"   • Precision: {precision:.1%}")
    print(f"   • Recall: {recall:.1%}")

exercise_jump_diffusion_pf()

In [ ]:
# Exercise 2: Multi-asset regime detection

def exercise_multivariate_regime_pf():
    """
    Exercise: Detect common regimes across multiple assets.
    
    Model: All assets share a common hidden regime.
    r_{i,t} = μ_{i,s_t} + σ_{i,s_t} * ε_{i,t}
    """
    
    # Parameters for 3 assets
    n_assets = 3
    asset_names = ['Equity', 'Bond', 'Commodity']
    
    # Regime 0: Risk-on (equity up, bond down, commodity up)
    # Regime 1: Risk-off (equity down, bond up, commodity down)
    mu = np.array([[0.001, -0.0003, 0.0008],   # Risk-on means
                   [-0.002, 0.0005, -0.001]])   # Risk-off means
    
    sigma = np.array([[0.012, 0.004, 0.015],    # Risk-on vols
                      [0.025, 0.006, 0.022]])    # Risk-off vols
    
    P = np.array([[0.97, 0.03],
                  [0.05, 0.95]])
    
    T = 400
    N = 500
    
    # Simulate
    np.random.seed(123)
    s = np.zeros(T, dtype=int)
    s[0] = 0
    
    for t in range(1, T):
        s[t] = np.random.choice([0, 1], p=P[s[t-1]])
    
    returns = np.zeros((T, n_assets))
    for t in range(T):
        for i in range(n_assets):
            returns[t, i] = mu[s[t], i] + sigma[s[t], i] * np.random.randn()
    
    # Multivariate particle filter
    particles = np.random.choice([0, 1], size=N, p=[0.5, 0.5])
    weights = np.ones(N) / N
    
    prob_riskoff = np.zeros(T)
    
    for t in range(T):
        if t > 0:
            new_particles = np.zeros(N, dtype=int)
            for i in range(N):
                new_particles[i] = np.random.choice([0, 1], p=P[particles[i]])
            particles = new_particles
        
        # Multivariate likelihood (product of univariate)
        likelihood = np.ones(N)
        for i in range(N):
            for j in range(n_assets):
                likelihood[i] *= stats.norm.pdf(returns[t, j], 
                                                 mu[particles[i], j],
                                                 sigma[particles[i], j])
        
        weights = weights * likelihood
        weights /= weights.sum()
        
        prob_riskoff[t] = np.sum(weights * particles)
        
        # Resample
        ess = 1.0 / np.sum(weights**2)
        if ess < N / 2:
            indices = np.random.choice(N, size=N, p=weights)
            particles = particles[indices]
            weights = np.ones(N) / N
    
    # Plot
    fig, axes = plt.subplots(n_assets + 2, 1, figsize=(14, 12))
    
    colors = ['blue', 'orange', 'green']
    for i in range(n_assets):
        cumret = np.cumsum(returns[:, i])
        axes[i].plot(cumret, color=colors[i], lw=1)
        # Shade risk-off periods
        in_riskoff = False
        start = 0
        for t in range(T):
            if s[t] == 1 and not in_riskoff:
                start = t
                in_riskoff = True
            elif s[t] == 0 and in_riskoff:
                axes[i].axvspan(start, t, alpha=0.2, color='red')
                in_riskoff = False
        if in_riskoff:
            axes[i].axvspan(start, T, alpha=0.2, color='red')
        axes[i].set_ylabel(asset_names[i])
        axes[i].set_title(f'{asset_names[i]} Cumulative Returns (Red = Risk-off)')
    
    # Regime detection
    axes[n_assets].fill_between(range(T), 0, s, alpha=0.3, color='red', 
                                 label='True Risk-off', step='mid')
    axes[n_assets].plot(prob_riskoff, 'b-', lw=1.5, label='P(Risk-off|data)', alpha=0.8)
    axes[n_assets].set_ylabel('Probability')
    axes[n_assets].set_title('Multivariate Regime Detection')
    axes[n_assets].legend()
    
    # All assets together
    for i in range(n_assets):
        axes[n_assets+1].plot(np.cumsum(returns[:, i]), color=colors[i], 
                               lw=1, label=asset_names[i])
    axes[n_assets+1].set_xlabel('Time')
    axes[n_assets+1].set_ylabel('Cumulative Return')
    axes[n_assets+1].set_title('All Assets Overlay')
    axes[n_assets+1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Metrics
    accuracy = np.mean((prob_riskoff > 0.5) == s)
    print(f"\n📊 Multivariate Regime Detection:")
    print(f"   • Accuracy: {accuracy:.1%}")
    print(f"   • Risk-off frequency (true): {np.mean(s):.1%}")
    print(f"   • Risk-off frequency (detected): {np.mean(prob_riskoff > 0.5):.1%}")

exercise_multivariate_regime_pf()

---

## 📝 Summary

### Key Takeaways

1. **Particle filters** solve filtering problems that Kalman filters cannot:
   - Nonlinear dynamics
   - Non-Gaussian noise
   - Discrete state spaces

2. **Core algorithm** (Bootstrap PF):
   - Predict: Propagate particles through state transition
   - Update: Reweight by observation likelihood
   - Resample: Avoid weight degeneracy

3. **Financial applications**:
   - Stochastic volatility estimation
   - Regime detection
   - Jump detection
   - Online parameter learning

4. **Practical considerations**:
   - Monitor ESS for particle degeneracy
   - Use systematic/stratified resampling
   - Consider auxiliary PF for better proposals
   - GPU parallelization for real-time applications

### Next Steps

- Day 6: Gaussian Processes for financial modeling
- Explore: Particle MCMC for parameter estimation
- Advanced: Rao-Blackwellized PF for mixed linear/nonlinear models

---

**References:**
1. Doucet, A., de Freitas, N., & Gordon, N. (2001). Sequential Monte Carlo Methods in Practice
2. Johannes, M., Korteweg, A., & Polson, N. (2014). Sequential Learning, Predictability, and Optimal Portfolio Returns
3. Pitt, M. K., & Shephard, N. (1999). Filtering via Simulation: Auxiliary Particle Filters
4. Andrieu, C., Doucet, A., & Holenstein, R. (2010). Particle Markov Chain Monte Carlo Methods